In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'countries.csv',
 'currency_details.csv',
 'fraudsters.csv',
 'fx_rates.csv',
 'message_from_the_data_team.txt',
 'Task 2.ipynb',
 'transactions.xls',
 'users.csv',
 '__MACOSX']

In [3]:
transactions=pd.read_excel('transactions.xls')
country=pd.read_csv('countries.csv')
currency_details=pd.read_csv('currency_details.csv')
fx_rate=pd.read_csv('fx_rates.csv')
fraudster=pd.read_csv('fraudsters.csv')

In [4]:
print(transactions.shape,transactions.USER_ID.nunique())

(65535, 12) 1420


In [5]:
transactions.isnull().sum()

Unnamed: 0               0
CURRENCY                 0
AMOUNT                   0
STATE                    0
CREATED_DATE             0
MERCHANT_CATEGORY    47002
MERCHANT_COUNTRY     18222
ENTRY_METHOD             0
USER_ID                  0
TYPE                     0
SOURCE                   0
ID                       0
dtype: int64

In [6]:
#Filtering out us rates
fx_us=fx_rate[fx_rate.base_ccy=='USD']

In [7]:
print(transactions.shape)
transactions=transactions.merge(fx_us,left_on='CURRENCY',right_on='ccy',how='left')
print(transactions.shape)

(65535, 12)
(65535, 15)


In [8]:
#checking if all the null conversions are because of USD
transactions[transactions.rate.isnull()]['CURRENCY'].value_counts()

USD    5973
Name: CURRENCY, dtype: int64

In [9]:
#filling it with NA
transactions['rate']=transactions['rate'].fillna(1)

In [10]:
#Getting in Exponenet
print(transactions.shape)
transactions=transactions.merge(currency_details[['currency','exponent']],
                                left_on='CURRENCY',right_on='currency',how='left')
print(transactions.shape)

(65535, 15)
(65535, 17)


In [11]:
transactions["AMOUNT_USD"]=((transactions["AMOUNT"])/(10**transactions["exponent"]))*(transactions["rate"])

In [12]:
transactions

,Unnamed: 0,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID,base_ccy,ccy,rate,currency,exponent,AMOUNT_USD
0,0,GBP,3738,COMPLETED,2015-10-11 09:05:43.016,bar,AUS,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470,USD,GBP,1.319906,GBP,2.0,49.338090
1,1,GBP,588,COMPLETED,2015-10-11 20:08:39.150,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5,USD,GBP,1.319906,GBP,2.0,7.761048
2,2,GBP,1264,COMPLETED,2015-10-11 11:37:40.908,NaN,UKR,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df,USD,GBP,1.319906,GBP,2.0,16.683613
3,3,GBP,66,REVERTED,2015-10-11 20:08:35.310,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32,USD,GBP,1.319906,GBP,2.0,0.871138
4,4,GBP,968,COMPLETED,2015-10-11 02:46:47.640,supermarket,NZL,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f,USD,GBP,1.319906,GBP,2.0,12.776691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,66213,GBP,0,DECLINED,2016-12-28 21:49:32.176,NaN,NLD,manu,ec4fd825-7167-450b-aa9f-0cbcf681978b,CARD_PAYMENT,GAIA,d19974bd-7561-40c1-aa4e-85f3ae2fe8da,USD,GBP,1.319906,GBP,2.0,0.000000
65531,66214,GBP,0,DECLINED,2016-12-28 21:49:32.121,NaN,NLD,manu,ec4fd825-7167-450b-aa9f-0cbcf681978b,CARD_PAYMENT,GAIA,dee61544-5103-4190-a8bc-b6eaf5e0e961,USD,GBP,1.319906,GBP,2.0,0.000000
65532,66215,EUR,97,REVERTED,2016-12-28 21:47:25.143,atm,GBR,manu,15fd8952-d6c5-4a1e-8192-9d4a7310fe72,CARD_PAYMENT,GAIA,83ecb8db-e39d-460e-a611-0682773e36ef,USD,EUR,1.177809,EUR,2.0,1.142475
65533,66216,EUR,24,COMPLETED,2016-12-28 21:43:39.410,NaN,POL,chip,c8a15f57-c18c-4af7-8c5e-b80f50f370c4,CARD_PAYMENT,GAIA,ea933c06-4f9c-4708-b488-e8fffc828781,USD,EUR,1.177809,EUR,2.0,0.282674


In [13]:
print(transactions['CREATED_DATE'].min())
print(transactions['CREATED_DATE'].max())

2015-07-08 18:24:11.565000
2016-12-31 23:03:10.839000


# Creating features at user level to understand fraud behaviour


# Mean, Max and Total trnasactions for each state per user

In [14]:
feature_state=transactions.groupby(['USER_ID','STATE']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_state.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_state.columns]
feature_state=feature_state.reset_index()
feature_state=feature_state.fillna(0)

# Mean, Max and Total trnasactions for each Type per USER

In [15]:
feature_type=transactions.groupby(['USER_ID','TYPE']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_type.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_type.columns]
feature_type=feature_type.reset_index()
feature_type=feature_type.fillna(0)

# Mean, Max and Total trnasactions for each SOURCE per USER

In [16]:
feature_source=transactions.groupby(['USER_ID','SOURCE']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_source.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_source.columns]
feature_source=feature_source.reset_index()
feature_source=feature_source.fillna(0)

# Mean, Max and Total trnasactions for each ENTRY_METHOD per USER

In [17]:
feature_entry=transactions.groupby(['USER_ID','ENTRY_METHOD']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_entry.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_entry.columns]
feature_entry=feature_entry.reset_index()
feature_entry=feature_entry.fillna(0)

# Merachant level features

In [18]:
transactions['MERCHANT_CATEGORY']=transactions['MERCHANT_CATEGORY'].fillna("Other_cat")

In [19]:
transactions.\
assign(count=1).\
groupby('MERCHANT_CATEGORY').\
agg({'count':sum}).\
assign(count_perc=lambda a:a['count']/sum(a['count'])).\
sort_values('count_perc',ascending=False).\
assign(count_perc_cumsum=lambda a:np.cumsum(a['count_perc'])).head(8)

,count,count_perc,count_perc_cumsum
MERCHANT_CATEGORY,,,
Other_cat,47002,0.717205,0.717205
point_of_interest,3869,0.059037,0.776242
atm,2199,0.033555,0.809796
supermarket,2011,0.030686,0.840482
restaurant,1525,0.023270,0.863752
bank,986,0.015045,0.878798
bar,788,0.012024,0.890822
cafe,703,0.010727,0.901549


# Taking top 5 merchange categories

In [20]:
transactions['merchant_cat']=np.where(transactions['MERCHANT_CATEGORY'].isin(['point_of_interest','atm','supermarket',
                                                                    'restaurant','bank']),transactions['MERCHANT_CATEGORY'],
                                                                            "Others")

# Mean, Max and Total trnasactions for each Merchant Categories per USER

In [21]:
feature_merchant_cat=transactions.groupby(['USER_ID','merchant_cat']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_merchant_cat.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_merchant_cat.columns]
feature_merchant_cat=feature_merchant_cat.reset_index()
feature_merchant_cat=feature_merchant_cat.fillna(0)

# Transactions in each quarter 

In [22]:
transactions['CREATED_DATE']=pd.to_datetime(transactions['CREATED_DATE'])
transactions['QUARTER_DATE']='Q'+(transactions['CREATED_DATE'].dt.quarter).astype(str)+"_"+(transactions['CREATED_DATE'].dt.year).astype(str)

# Mean, Max and Total trnasactions for each quarter method per USER

In [23]:
feature_quarter=transactions.groupby(['USER_ID','QUARTER_DATE']).agg({'AMOUNT_USD':[np.mean,np.sum,np.max]}).unstack(level=1)
feature_quarter.columns=[i[0]+'_'+i[2]+'_'+i[1] for i in feature_quarter.columns]
feature_quarter=feature_quarter.reset_index()
feature_quarter=feature_quarter.fillna(0)

# merging the features

In [24]:
df_users=transactions[['USER_ID']].drop_duplicates()
df_users=df_users.merge(feature_state,on='USER_ID',how='left')
df_users=df_users.merge(feature_type,on='USER_ID',how='left')
df_users=df_users.merge(feature_source,on='USER_ID',how='left')
df_users=df_users.merge(feature_entry,on='USER_ID',how='left')
df_users=df_users.merge(feature_merchant_cat,on='USER_ID',how='left')
df_users=df_users.merge(feature_quarter,on='USER_ID',how='left')

# Getting the fraud flag

In [25]:
df_users=df_users.merge(fraudster[['user_id']].\
                        rename(columns={"user_id":"USER_ID"}).\
                        assign(fraud_flag=1),on='USER_ID',how='left')

In [26]:
df_users['fraud_flag']=df_users['fraud_flag'].fillna(0)

In [27]:
set(fraudster.user_id).\
intersection(set(transactions.USER_ID))

{'25c2ecb3-5ec7-4fa6-8fc3-bbcf3a691217',
 '36c414e8-da9c-4c38-9a01-e0932263a300',
 '6eb51e3f-6e8b-45ee-bf01-82495df9c930',
 'eafda277-d957-4c5f-9b1c-a0d04ecb8241'}

# Building a Random Forest to indentify Fraud

In [28]:
feature_list=list(set(df_users.columns).difference(set(['USER_ID','fraud_flag'])))
target_variable='fraud_flag'

In [29]:
from sklearn.ensemble import RandomForestClassifier
mod=RandomForestClassifier(n_estimators=500,random_state=10,max_depth=8)

In [30]:
mod.fit(df_users[feature_list],
       df_users[target_variable])

RandomForestClassifier(max_depth=8, n_estimators=500, random_state=10)

In [31]:
df_users['fraud_score']=mod.predict_proba(df_users[feature_list])[:,1]

In [42]:
df_users.sort_values('fraud_score',ascending=False).head()

,USER_ID,AMOUNT_USD_CANCELLED_mean,AMOUNT_USD_COMPLETED_mean,AMOUNT_USD_DECLINED_mean,AMOUNT_USD_FAILED_mean,AMOUNT_USD_PENDING_mean,AMOUNT_USD_RECORDED_mean,AMOUNT_USD_REVERTED_mean,AMOUNT_USD_CANCELLED_sum,AMOUNT_USD_COMPLETED_sum,...,AMOUNT_USD_Q4_2016_sum,AMOUNT_USD_Q1_2016_amax,AMOUNT_USD_Q2_2016_amax,AMOUNT_USD_Q3_2015_amax,AMOUNT_USD_Q3_2016_amax,AMOUNT_USD_Q4_2015_amax,AMOUNT_USD_Q4_2016_amax,fraud_flag,fraud_score,fraud_buckets
305,eafda277-d957-4c5f-9b1c-a0d04ecb8241,0.0,0.000000,0.000000,11879.154963,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,11879.154963,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.660000,"(0.024, 0.66]"
1240,25c2ecb3-5ec7-4fa6-8fc3-bbcf3a691217,0.0,0.000000,0.013199,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.026398,0.000000,0.0,0.000000,0.0,0.000000,0.013199,1.0,0.653239,"(0.024, 0.66]"
129,6eb51e3f-6e8b-45ee-bf01-82495df9c930,0.0,33.245454,37.386303,110.872113,0.0,0.0,6.256355,0.0,2061.218173,...,0.000000,461.967137,0.0,0.000000,0.0,65.995305,0.000000,1.0,0.646000,"(0.024, 0.66]"
107,36c414e8-da9c-4c38-9a01-e0932263a300,0.0,0.000000,0.000000,529.282349,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,530.602255,0.0,0.000000,0.000000,1.0,0.616667,"(0.024, 0.66]"
1119,add354a8-edd3-4dc3-8b28-42b924908999,0.0,0.000000,0.011778,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.011778,0.000000,0.0,0.000000,0.0,0.000000,0.011778,0.0,0.160239,"(0.024, 0.66]"


In [33]:
feature_imp=pd.DataFrame({'features':feature_list,
    'Feature_Imp':mod.feature_importances_})
feature_imp

,features,Feature_Imp
0,AMOUNT_USD_P2P_mean,0.000976
1,AMOUNT_USD_bank_mean,0.000000
2,AMOUNT_USD_FAILED_sum,0.022185
3,AMOUNT_USD_CRONUS_amax,0.003102
4,AMOUNT_USD_INTERNAL_mean,0.000340
...,...,...
106,AMOUNT_USD_CARD_PAYMENT_amax,0.019006
107,AMOUNT_USD_CANCELLED_mean,0.000611
108,AMOUNT_USD_Q1_2016_mean,0.035602
109,AMOUNT_USD_supermarket_mean,0.008082


# grouping to see similar fraudsters

In [34]:
df_users['fraud_buckets']=pd.qcut(df_users['fraud_score'],100,duplicates='drop')

In [35]:
df_users.groupby('fraud_buckets').\
agg({'USER_ID':len,
    'fraud_flag':sum,
'fraud_score':np.mean})

,USER_ID,fraud_flag,fraud_score
fraud_buckets,,,
"(-0.001, 0.000105]",1204,0.0,0.000001
"(0.000105, 0.000133]",15,0.0,0.000133
"(0.000133, 0.002]",98,0.0,0.002000
"(0.002, 0.00356]",3,0.0,0.002480
"(0.00356, 0.004]",26,0.0,0.004000
"(0.004, 0.006]",19,0.0,0.006000
"(0.006, 0.008]",16,0.0,0.007757
"(0.008, 0.0112]",10,0.0,0.010000
"(0.0112, 0.024]",15,0.0,0.016140


In [37]:
df_users[['USER_ID','fraud_flag','fraud_score']].sort_values('fraud_score')

,USER_ID,fraud_flag,fraud_score
0,7285c1ec-31d0-4022-b311-0ad9227ef7f4,0.0,0.000000
926,ae3507be-50f1-402c-96da-ae1ee2741d83,0.0,0.000000
925,fde8dd5a-5c34-4bc0-ac65-9014f936ec40,0.0,0.000000
924,ecfc7e59-6987-466c-bcc7-b64dbbd0f9eb,0.0,0.000000
923,7bd76ab3-6e26-4553-ba0f-376e3fe903c4,0.0,0.000000
...,...,...,...
1119,add354a8-edd3-4dc3-8b28-42b924908999,0.0,0.160239
107,36c414e8-da9c-4c38-9a01-e0932263a300,1.0,0.616667
129,6eb51e3f-6e8b-45ee-bf01-82495df9c930,1.0,0.646000
1240,25c2ecb3-5ec7-4fa6-8fc3-bbcf3a691217,1.0,0.653239


In [49]:
df_users.head(5)

,USER_ID,AMOUNT_USD_CANCELLED_mean,AMOUNT_USD_COMPLETED_mean,AMOUNT_USD_DECLINED_mean,AMOUNT_USD_FAILED_mean,AMOUNT_USD_PENDING_mean,AMOUNT_USD_RECORDED_mean,AMOUNT_USD_REVERTED_mean,AMOUNT_USD_CANCELLED_sum,AMOUNT_USD_COMPLETED_sum,...,AMOUNT_USD_Q4_2016_sum,AMOUNT_USD_Q1_2016_amax,AMOUNT_USD_Q2_2016_amax,AMOUNT_USD_Q3_2015_amax,AMOUNT_USD_Q3_2016_amax,AMOUNT_USD_Q4_2015_amax,AMOUNT_USD_Q4_2016_amax,fraud_flag,fraud_score,fraud_buckets
0,7285c1ec-31d0-4022-b311-0ad9227ef7f4,0.0,58.744047,0.000000,0.000000,0.000000,0.0,110.872113,0.0,4053.339261,...,0.000000,0.000000,13.199061,409.170893,197.985916,329.976527,0.000000,0.0,0.000000,"(-0.001, 0.000105]"
1,20100a1d-12bc-41ed-a5e1-bc46216e9696,0.0,82.279901,66.835643,0.000000,7.259484,0.0,122.315699,0.0,51754.057750,...,3934.120896,923.934275,1121.920191,26.398122,2129.034949,2639.812214,263.981221,0.0,0.002000,"(0.000133, 0.002]"
2,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,0.0,49.081920,70.404892,329.976527,0.000000,0.0,43.602155,0.0,38234.815724,...,9977.949007,158.388733,461.967137,659.953054,400.855485,659.953054,296.978874,0.0,0.000000,"(-0.001, 0.000105]"
3,821014c5-af06-40ff-91f4-77fe7667809f,0.0,31.509894,71.783094,90.633553,0.000000,0.0,0.695151,0.0,8350.122005,...,0.000000,131.990611,56.755963,263.981221,724.628453,359.727210,0.000000,0.0,0.014000,"(0.0112, 0.024]"
4,fbe6dfd9-96de-4fde-af16-32e8a0bb7a25,0.0,0.000000,15.766278,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,21.659659,0.000000,0.0,0.000105,"(-0.001, 0.000105]"


In [48]:
df_users.loc[df_users['fraud_score'] >= 0.160230]

,USER_ID,AMOUNT_USD_CANCELLED_mean,AMOUNT_USD_COMPLETED_mean,AMOUNT_USD_DECLINED_mean,AMOUNT_USD_FAILED_mean,AMOUNT_USD_PENDING_mean,AMOUNT_USD_RECORDED_mean,AMOUNT_USD_REVERTED_mean,AMOUNT_USD_CANCELLED_sum,AMOUNT_USD_COMPLETED_sum,...,AMOUNT_USD_Q4_2016_sum,AMOUNT_USD_Q1_2016_amax,AMOUNT_USD_Q2_2016_amax,AMOUNT_USD_Q3_2015_amax,AMOUNT_USD_Q3_2016_amax,AMOUNT_USD_Q4_2015_amax,AMOUNT_USD_Q4_2016_amax,fraud_flag,fraud_score,fraud_buckets
107,36c414e8-da9c-4c38-9a01-e0932263a300,0.0,0.000000,0.000000,529.282349,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,530.602255,0.0,0.000000,0.000000,1.0,0.616667,"(0.024, 0.66]"
129,6eb51e3f-6e8b-45ee-bf01-82495df9c930,0.0,33.245454,37.386303,110.872113,0.0,0.0,6.256355,0.0,2061.218173,...,0.000000,461.967137,0.0,0.000000,0.0,65.995305,0.000000,1.0,0.646000,"(0.024, 0.66]"
305,eafda277-d957-4c5f-9b1c-a0d04ecb8241,0.0,0.000000,0.000000,11879.154963,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,11879.154963,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.660000,"(0.024, 0.66]"
1119,add354a8-edd3-4dc3-8b28-42b924908999,0.0,0.000000,0.011778,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.011778,0.000000,0.0,0.000000,0.0,0.000000,0.011778,0.0,0.160239,"(0.024, 0.66]"
1240,25c2ecb3-5ec7-4fa6-8fc3-bbcf3a691217,0.0,0.000000,0.013199,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.026398,0.000000,0.0,0.000000,0.0,0.000000,0.013199,1.0,0.653239,"(0.024, 0.66]"
